## Day 14: Regolith Reservoir

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.org/github/mazharenko/AoC-2022/tree/HEAD/notebooks/day/puzzle.ipynb)

### Parsing

In [15]:
#!value --name sampleRaw 
498,4 -> 498,6 -> 496,6
503,4 -> 502,4 -> 502,9 -> 494,9

In [16]:
#r "nuget:Farkle, 6.3.2"
open Farkle
open Farkle.Builder

Installed Packages Farkle, 6.3.2

In [17]:

#load "../common/common.fsx"

let private number = Terminals.int "Number"
let private point = "Point" ||= [
    !@ number .>> "," .>>. number => fun x y -> Point(x,y)
]
let private path = nonterminal "Path"
path.SetProductions(
    !@ path .>> "->" .>>. point => (fun (from::rest) to' -> [to'..(Point.dir (from - to'))..from] @ rest),
    !@ point => fun p -> [ p ]
)
let private parser = RuntimeFarkle.build path

let private parsePath s = 
    RuntimeFarkle.parseString parser s 
    |> Result.get

let parse s = Pattern1.read parsePath s |> Seq.collect id



In [18]:
type Cell = 
    | Rock
    | Sand
    | FallingSand
    | Empty

In [19]:
type State = { Cells : Cell [,]; Spawn: Point; FallingSand : Point option }

let stateFromInput s = 
    let rocks = parse s 
    let spawn = Point (500, 0)
    let minx = 
        rocks |> Seq.map Point.x |> Seq.min 
        |> min (Point.x spawn)
    let miny = 
        rocks |> Seq.map Point.y |> Seq.min 
        |> min (Point.y spawn)
    let maxx = 
        rocks |> Seq.map Point.x |> Seq.max 
        |> max (Point.x spawn)
    let maxy = 
        rocks |> Seq.map Point.y |> Seq.max 
        |> max (Point.y spawn)

    let m = Array2D.createBased minx miny (maxx - minx + 1) (maxy - miny + 1) Empty
    rocks
    |> Seq.iter (
        fun p -> 
            Array2D.set m (Point.x p) (Point.y p) Rock
    )
    { Cells = m; Spawn = spawn; FallingSand = None }


In [20]:
#load "../common/matrixFormatting.fsx"

In [21]:

let displayState state = 
    let color cell = 
        match cell with 
        | Rock -> System.Drawing.Color.Gray
        | Sand -> System.Drawing.Color.Yellow
        | _ -> System.Drawing.Color.Transparent

    let newCells = state.Cells |> Array2D.rebase |> Array2D.transpose
    let displayable = toDisplayable color (newCells)
    displayable

In [22]:
#!share sampleRaw --from value
stateFromInput sampleRaw |> displayState

### Part 1

In [30]:
type private Free = 
    | Occupied
    | Unoccupied
    | OutOfMap

let private isFree point s =
    match Array2D.tryGet (Point.x point) (Point.y point) s.Cells with
    | Some value -> 
        match value with
        | Empty -> Unoccupied
        | _ -> Occupied
    | None -> OutOfMap

let rec private fall' state = 
    match state.FallingSand with
    | None -> 
        if (Array2D.tryGet (Point.x state.Spawn) (Point.y state.Spawn) state.Cells = Some Empty)
        then (fall' {state with FallingSand = state.Spawn |> Some})
        else state
    | Some sand -> 
        let sandToCandidates = 
            [
                sand + Point(0,1)
                sand + Point(-1,1)
                sand + Point(1,1)
            ] |> List.map (fun p -> p, isFree p state)

        if sandToCandidates |> List.forall (snd >> (=)Occupied)
        then 
            state.Cells[Point.x sand, Point.y sand] <- Sand
            (fall' {state with FallingSand = None})
        elif sandToCandidates |> List.exists (snd >> (=)OutOfMap)
        then 
            state
        else
            let sandTo = sandToCandidates |> List.find (snd >> (=)Unoccupied) |> fst
            state.Cells[Point.x sand, Point.y sand] <- Empty
            state.Cells[Point.x sandTo, Point.y sandTo] <- FallingSand
            fall' {state with FallingSand = Some sandTo}

let fall state = 
    fall' { state with Cells = Array2D.copy state.Cells }

In [31]:
(stateFromInput sampleRaw
|> fall).Cells |> Array2D.toSeq
|> Seq.filter ((=)Sand)
|> Seq.length

24

In [32]:
stateFromInput sampleRaw
|> fall |> displayState

In [33]:
#!value --name actualRaw --from-file ./data_actual.txt

In [56]:
#!share actualRaw --from value

(stateFromInput actualRaw |> fall).Cells
|> Array2D.toSeq
|> Seq.filter ((=)Sand)
|> Seq.length

737

In [57]:
stateFromInput actualRaw |> fall |> displayState |> withSettings { Width = 600 }

### Part 2

In [48]:
let withFloor state = 
    let floory = 
        state.Cells
        |> Array2D.indexed
        |> Array2D.toSeq
        |> Seq.filter (fun ((x,y), value) -> value = Rock)
        |> Seq.map (fun ((_,y), _) -> y)
        |> Seq.max
        |> (+) 2
    let floorxfrom = Point.x state.Spawn - floory - 5
    let floorxto = Point.x state.Spawn + floory + 5
    let newCells = 
        Array2D.initBased 
            floorxfrom 
            (Array2D.base2 state.Cells)
            (floorxto - floorxfrom + 1)
            (Array2D.length2 state.Cells + 3)
            (fun i j -> state.Cells |> Array2D.tryGet i j |> Option.defaultValue Empty)
    
    let floor = 
        [
            Point(floorxfrom, floory) .. Point(1,0) .. Point(floorxto, floory)
        ]
    floor 
    |> Seq.iter (fun (Point(x,y)) -> newCells[x,y] <- Rock )

    { state with Cells = newCells }
    

In [50]:
(stateFromInput sampleRaw) |> withFloor |> displayState |> withSettings { Width = 300 }

In [51]:
(stateFromInput sampleRaw |> withFloor
|> fall).Cells |> Array2D.toSeq
|> Seq.filter ((=)Sand)
|> Seq.length

93

In [54]:
stateFromInput sampleRaw |> withFloor
|> fall |> displayState |> withSettings { Width = 300 }

In [69]:
(stateFromInput actualRaw |> withFloor |> fall).Cells
|> Array2D.toSeq
|> Seq.filter ((=)Sand)
|> Seq.length

28145

In [68]:
stateFromInput actualRaw |> withFloor |> fall
|> displayState |> withSettings { Width = 1200 }